## Dataset 

Esse dataset contém 5 colunas:
* **ID:** ID do tweet
* **Keyword**: Uma palavra chave do tweet (pode estar em branco)
* **Location:** A localização de onde o tweet foi postado(pode estar em branco)
* **Text:** coluna onde estão os tweets
* **Target:** mostra se o tweet fala de uma **desastre real (1)** ou **falso (0)**


In [1]:
import pandas as pd
import numpy as np
import nltk 

Vamos dar uma olhada no nosso Dataset:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = df_bert = pd.read_csv('/content/drive/Shared drives/Grupo Turing/P&D/Áreas de Foco/NLP/Materiais/Aulas 2020/Aula Transfer Learning Word2Vec/datasets/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Pré-processamento

Antes de continuarmos a análise vamos pré-processar nossos tweets, peguei parte do código do pré-processamento que a Camilala, a Lu e a Ju fizeram para a aulinha de EDA dos tweets do corona, obrigada meninas <3 

In [ ]:
import nltk
import spacy
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
spc = spacy.load('en')

In [ ]:
df = df.drop_duplicates('id', keep="first")
df.dropna(inplace=True)

In [ ]:
df.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [ ]:
def limpa_tweets(texto):
   # Remove links e '<user>' e converte para letra minuscula
    sem_links = re.sub(r"http\S+", "", texto.lower())
    sem_user = re.sub("<user>", "", sem_links)

    # Remove caracteres que nao sao letras e tokeniza
    tokens =  re.findall(r'\b[A-zÀ-úü]+\b', sem_user)
    
    # Remove stopwords
    stopw = nltk.corpus.stopwords.words('english')
    stopw_set = set(stopw)

    sem_stopwords = [w for w in tokens if w not in stopw_set]

    junta_texto = " ".join(sem_stopwords)

    # Instanciando o objeto spacy
    spc_letras =  spc(junta_texto)

    # Lemmização 
    lemma = [token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras]
  
    # juntando os tokens 
    tweets_processados = " ".join(lemma)
  
    return sem_stopwords

In [ ]:
df['text'] = df['text'].apply(limpa_tweets)

In [ ]:
df.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,"[bbcmtd, wholesale, markets, ablaze]",1
32,49,ablaze,Est. September 2012 - Bristol,"[always, try, bring, heavy, metal, rt]",0
33,50,ablaze,AFRICA,"[africanbaze, breaking, news, nigeria, flag, s...",1
34,52,ablaze,"Philadelphia, PA","[crying, set, ablaze]",0
35,53,ablaze,"London, UK","[plus, side, look, sky, last, night, ablaze]",0


## Word2Vec

In [ ]:
from gensim.models import KeyedVectors

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
import keras.backend as K

### Layer de Embedding do Keras

In [ ]:
vocab_len = 10
embedding_dim = 5
embedding_matrix = np.random.randn(vocab_len, embedding_dim)

embedding_example = Embedding(vocab_len, embedding_dim, 
                              embeddings_initializer = Constant(embedding_matrix),
                              trainable = False)

In [ ]:
# One hot equivalente = [1, 0, 0, 0, 0, 0, 0, 0, 0]
#               indices: 0  1  2  3  5  6  7  8  9
embedding_example(0)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.38147095, -0.59538394,  0.65744424,  0.5225975 ,  0.07095776],
      dtype=float32)>

In [ ]:
# One hot equivalente = [0, 1, 0, 0, 0, 0, 0, 0, 0]
#               indices: 0  1  2  3  5  6  7  8  9
embedding_example(1)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-0.46625623,  0.4935111 , -0.4437881 ,  0.26713082,  0.1052963 ],
      dtype=float32)>

In [ ]:
embedding_example(np.array([0, 1, 2]))

<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[-0.37037277, -0.18537672,  0.18003024,  0.13673331, -2.0339165 ,
         0.25801376, -0.69475514, -0.6896385 ,  0.37915477, -1.2440747 ],
       [-0.39282772,  0.07648234, -0.48073792, -2.2546585 ,  0.04022843,
         1.5837353 ,  0.09144673,  1.1881027 , -1.056005  ,  1.1102625 ],
       [-1.7462479 ,  0.81345886, -0.11882789,  0.83243835,  0.11795305,
        -0.6272579 , -0.7170191 , -0.00369342, -1.2439246 , -0.47591078]],
      dtype=float32)>

### Modelagem de redes

In [ ]:
def create_features(tweets, max_len=50):
  """
  Função que substitui palavras por ids e adiciona padding na lista.

  Parâmetros
    tweets(array): Lista com tweets tokenizados
    max_len(int): Tamanho máximos dos tweets

  Retorna
    tweet_pad(pd.DataFrame): Lista de tweets processados (com ids e paddings)
    tokenizer(Tokenizer[tf]): Objeto do tokenizador do tensorflow
  """

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(tweets)
  sequences = tokenizer.texts_to_sequences(tweets)

  tweet_pad = pad_sequences(sequences, maxlen=max_len, truncating='post', padding='post')

  return tweet_pad, tokenizer

In [ ]:
tweet_pad, tokenizer = create_features(df['text'])

X_train, X_test, y_train, y_test = train_test_split(tweet_pad, df['target'].values, test_size=0.2)

In [ ]:
tweet_pad[1]

array([213, 730, 731, 570, 913,  45,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
tokenizer.index_word.get(213)

'always'

In [ ]:
def create_embedding_layer(embedding, tokenizer, embedding_dim, trainable=True):
 """
 Função que instância layer de embedding do tensorflow

 Parâmtros
    embedding(gensim): Objeto do gensim
    tokenizer(Tokenizer[tf]): Objeto do tokenizador do tensorflow, para identificar os ids gerados
    embedding_dim(int): Dimensão do embedding
    trainable(bool): Habilita a modificação dos pesos da layer de embedding

  Retorna
      embedding_layer(Embedding): Layer de embedding gerada

 """


  words_ids = tokenizer.word_index

  num_words = len(words_ids) + 1
  embedding_matrix = np.zeros((num_words, embedding_dim))

  for word, i in words_ids.items():
    if i < num_words:
      embedding_vector = embedding.get_vector(word)

      if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(num_words, embedding_dim, 
                                embeddings_initializer = Constant(embedding_matrix),
                                trainable = trainable)

    return embedding_layer

In [ ]:
def mount_lstm(embedding_layer):
  """
  Instância uma LSTM a partir da layer de embedding

  Parâmetros
      embedding_layer(Embedding): Layer de embedding gerada

  Retorna
      model(Sequential): modelo de rede neural
  """
  
  model = Sequential()

  model.add(embedding_layer)
  model.add(SpatialDropout1D(0.3))
  model.add(LSTM(40, dropout=0.3, recurrent_dropout=0.2))
  model.add(Dense(1, activation='sigmoid'))

  optimzer = Adam(learning_rate=8e-5)

  model.compile(loss='binary_crossentropy', optimizer=optimzer, metrics=['accuracy'])

  return model

def evaluate_neural_net(model, X_test, y_test, threshold = 0.5):

  y_hat = model.predict(X_test)

  y_hat_bool = y_hat > threshold

  print(classification_report(y_test, y_hat_bool))

### Treinamento com diferentes word embeddings

Datasets e word2vec disponíveis: https://github.com/RaRe-Technologies/gensim-data

#### Word2Vec de notiícias do Google

In [ ]:
import gensim.downloader as api
google_300d_embedding = api.load('word2vec-google-news-300')

[=================================================-] 99.1% 1648.0/1662.8MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Google 300D trainable=False

In [ ]:
google_300d_layer = create_embedding_layer(google_300d_embedding, tokenizer, embedding_dim=300, max_len=50, trainable=False)
google_lstm = mount_lstm(google_300d_layer)

In [ ]:
google_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
127/127 [==============================] - 11s 87ms/step - loss: 0.6914 - accuracy: 0.5595 - val_loss: 0.6887 - val_accuracy: 0.5679
Epoch 2/10
127/127 [==============================] - 11s 85ms/step - loss: 0.6860 - accuracy: 0.5677 - val_loss: 0.6839 - val_accuracy: 0.5679
Epoch 3/10
127/127 [==============================] - 11s 84ms/step - loss: 0.6844 - accuracy: 0.5677 - val_loss: 0.6838 - val_accuracy: 0.5679
Epoch 4/10
127/127 [==============================] - 11s 85ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6838 - val_accuracy: 0.5679
Epoch 5/10
127/127 [==============================] - 11s 85ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6837 - val_accuracy: 0.5679
Epoch 6/10
127/127 [==============================] - 11s 85ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6836 - val_accuracy: 0.5679
Epoch 7/10
127/127 [==============================] - 11s 85ms/step - loss: 0.6839 - accuracy: 0.5677 - val_loss: 0.6835 - val_accuracy:

In [ ]:
evaluate_neural_net(google_lstm, X_test, y_test)

              precision    recall  f1-score   support

           0       0.57      1.00      0.72       577
           1       0.00      0.00      0.00       439

    accuracy                           0.57      1016
   macro avg       0.28      0.50      0.36      1016
weighted avg       0.32      0.57      0.41      1016



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Google 300D trainable=True

In [ ]:
google_300d_layer_trainable = create_embedding_layer(google_300d_embedding, tokenizer, embedding_dim=300, max_len=50, trainable=True)
google_lstm_trainable = mount_lstm(google_300d_layer_trainable)

In [ ]:
google_lstm_trainable.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
127/127 [==============================] - 16s 127ms/step - loss: 0.6870 - accuracy: 0.5677 - val_loss: 0.6839 - val_accuracy: 0.5679
Epoch 2/10
127/127 [==============================] - 16s 123ms/step - loss: 0.6841 - accuracy: 0.5677 - val_loss: 0.6838 - val_accuracy: 0.5679
Epoch 3/10
127/127 [==============================] - 15s 122ms/step - loss: 0.6838 - accuracy: 0.5677 - val_loss: 0.6835 - val_accuracy: 0.5679
Epoch 4/10
127/127 [==============================] - 15s 122ms/step - loss: 0.6772 - accuracy: 0.5677 - val_loss: 0.6344 - val_accuracy: 0.5679
Epoch 5/10
127/127 [==============================] - 16s 122ms/step - loss: 0.4444 - accuracy: 0.8314 - val_loss: 0.5429 - val_accuracy: 0.7510
Epoch 6/10
127/127 [==============================] - 15s 122ms/step - loss: 0.3159 - accuracy: 0.8994 - val_loss: 0.6210 - val_accuracy: 0.7195
Epoch 7/10
127/127 [==============================] - 16s 124ms/step - loss: 0.2474 - accuracy: 0.9237 - val_loss: 0.5903 - val_ac

In [ ]:
evaluate_neural_net(google_lstm_trainable, X_test, y_test)

              precision    recall  f1-score   support

           0       0.82      0.77      0.79       577
           1       0.72      0.77      0.74       439

    accuracy                           0.77      1016
   macro avg       0.77      0.77      0.77      1016
weighted avg       0.77      0.77      0.77      1016



#### Word2Vec do Twitter

In [ ]:
twitter_embedding = KeyedVectors.load_word2vec_format('/content/drive/Shared drives/Grupo Turing/P&D/Áreas de Foco/NLP/Materiais/Aulas 2020/Aula Transfer Learning Word2Vec/word_embeddings/glove.twitter.27B.100d.txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Twitter trainable=False

In [ ]:
twitter_layer = create_embedding_layer(twitter_embedding, tokenizer, embedding_dim=100, trainable = False)
twitter_lstm = mount_lstm(twitter_layer)

In [ ]:
twitter_lstm.fit(X_train, y_train, epochs=7, validation_data = (X_test,y_test))

Epoch 1/7
127/127 [==============================] - 11s 85ms/step - loss: 0.6906 - accuracy: 0.5677 - val_loss: 0.6874 - val_accuracy: 0.5679
Epoch 2/7
127/127 [==============================] - 11s 84ms/step - loss: 0.6855 - accuracy: 0.5677 - val_loss: 0.6839 - val_accuracy: 0.5679
Epoch 3/7
127/127 [==============================] - 10s 82ms/step - loss: 0.6841 - accuracy: 0.5677 - val_loss: 0.6838 - val_accuracy: 0.5679
Epoch 4/7
127/127 [==============================] - 10s 82ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6837 - val_accuracy: 0.5679
Epoch 5/7
127/127 [==============================] - 10s 81ms/step - loss: 0.6839 - accuracy: 0.5677 - val_loss: 0.6836 - val_accuracy: 0.5679
Epoch 6/7
127/127 [==============================] - 10s 82ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6835 - val_accuracy: 0.5679
Epoch 7/7
127/127 [==============================] - 10s 82ms/step - loss: 0.6839 - accuracy: 0.5677 - val_loss: 0.6833 - val_accuracy: 0.5679

In [ ]:
evaluate_neural_net(twitter_lstm, X_test, y_test)

              precision    recall  f1-score   support

           0       0.57      1.00      0.72       577
           1       0.00      0.00      0.00       439

    accuracy                           0.57      1016
   macro avg       0.28      0.50      0.36      1016
weighted avg       0.32      0.57      0.41      1016



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Twitter trainable=True

In [ ]:
twitter_layer_trainable = create_embedding_layer(twitter_embedding, tokenizer, embedding_dim=100, trainable = True)
twitter_lstm_trainable = mount_lstm(twitter_layer_trainable)

In [ ]:
twitter_lstm_trainable.fit(X_train, y_train, epochs=7, validation_data = (X_test,y_test))

Epoch 1/7
127/127 [==============================] - 13s 104ms/step - loss: 0.6895 - accuracy: 0.5679 - val_loss: 0.6859 - val_accuracy: 0.5679
Epoch 2/7
127/127 [==============================] - 13s 100ms/step - loss: 0.6843 - accuracy: 0.5677 - val_loss: 0.6838 - val_accuracy: 0.5679
Epoch 3/7
127/127 [==============================] - 13s 100ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6839 - val_accuracy: 0.5679
Epoch 4/7
127/127 [==============================] - 13s 101ms/step - loss: 0.6840 - accuracy: 0.5677 - val_loss: 0.6837 - val_accuracy: 0.5679
Epoch 5/7
127/127 [==============================] - 13s 101ms/step - loss: 0.6831 - accuracy: 0.5677 - val_loss: 0.6826 - val_accuracy: 0.5679
Epoch 6/7
127/127 [==============================] - 13s 102ms/step - loss: 0.5893 - accuracy: 0.6713 - val_loss: 0.5533 - val_accuracy: 0.7569
Epoch 7/7
127/127 [==============================] - 13s 101ms/step - loss: 0.3734 - accuracy: 0.8723 - val_loss: 0.5325 - val_accuracy:

In [ ]:
evaluate_neural_net(twitter_lstm_trainable, X_test, y_test)

              precision    recall  f1-score   support

           0       0.80      0.82      0.81       577
           1       0.75      0.73      0.74       439

    accuracy                           0.78      1016
   macro avg       0.77      0.77      0.77      1016
weighted avg       0.78      0.78      0.78      1016



## Transformers

In [4]:
!pip install transformers

     |████████████████████████████████| 1.1MB 5.2MB/s 
     |████████████████████████████████| 1.1MB 17.1MB/s 
     |████████████████████████████████| 890kB 28.8MB/s 
     |████████████████████████████████| 3.0MB 44.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=33a5697b88813c3ed6bf8651afa52d2f5319683320c44bc13a2e84c8250ea62b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
from transformers import AutoTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from keras.layers import Dense

bert_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def bert_encode(tweets, Tokenizer, maximum_len):
  """
  Tokeniza, converte texto para id do vocabulário, constroi sequência e cria vetor com máscara de atenção.
  """
  input_ids = []
  attention_masks = []
  
  for tweet in tweets:
      encoded = Tokenizer.encode_plus(tweet,
                                      add_special_tokens=True,
                                      truncation=True,
                                      max_length=maximum_len,
                                      pad_to_max_length=True,
                                      return_attention_mask=True)
      
      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
        
  return np.array(input_ids), np.array(attention_masks)

def mount_bert_input(tweets, targets, Tokenizer, maximum_len=128, test_size=0.3):
  features = bert_encode(tweets, Tokenizer, maximum_len)

  return features, targets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_bert['text'], df_bert['target'], test_size = 0.10, random_state=42)

X_train, y_train = mount_bert_input(X_train, y_train, bert_tokenizer)
X_test, y_test = mount_bert_input(X_test, y_test, bert_tokenizer);

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.BinaryCrossentropy()
bert.compile(optimizer=optimizer, loss='binary_crossentropy')

In [ ]:
bert.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
bert.fit(X_train, y_train, epochs=3)

Epoch 1/3
215/215 [==============================] - 102s 476ms/step - loss: 0.5743
Epoch 2/3
215/215 [==============================] - 102s 477ms/step - loss: 0.5158
Epoch 3/3
215/215 [==============================] - 102s 476ms/step - loss: 0.4886


In [ ]:
y_hat = bert.predict(X_test)

In [ ]:
y_hat_bool = []
for i in y_hat[0]:
  y_hat_bool.append(i[0] > 0.5)

In [ ]:
print(classification_report(y_hat_bool, y_test))

              precision    recall  f1-score   support

       False       0.91      0.81      0.86       479
        True       0.73      0.87      0.79       283

    accuracy                           0.83       762
   macro avg       0.82      0.84      0.82       762
weighted avg       0.84      0.83      0.83       762

